In [ ]:
import scipp as sc
import plopp as pp
from beamlime.offline.proto import BeamLimeApplicationManager
from beamlime.logging import get_logger
import logging
from beamlime.logging.handlers import BeamlimeStreamHandler
from beamlime.logging.formatters import BeamlimeFormatter
%matplotlib widget

In [ ]:
logger = sc.get_logger()
logger.setLevel('INFO')
widget_handler = sc.logging.make_widget_handler()
logger.addHandler(widget_handler)

In [ ]:
sc.display_logs()

In [ ]:
# get_logger().addHandler(BeamlimeStreamHandler(header=True))
# get_logger().setLevel('INFO')
# manager = BeamLimeApplicationManager(save_log=True)
manager = BeamLimeApplicationManager(logger=logger)

In [ ]:
import asyncio
loop = asyncio.get_event_loop()

for coroutine in manager.coroutines:
    asyncio.run_coroutine_threadsafe(coroutine, loop)
    
# manager.create_task()

In [ ]:
manager.app_instances['visualization']._instances[0].figs['heatmap']